In [2]:
import os
import numpy as np
import pandas as pd

In [41]:
# https://jalammar.github.io/illustrated-transformer/
# for theory of transfer learning

In [3]:
os.getcwd()

'/content'

In [4]:
df = pd.read_csv("/content/spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [6]:
df_ham = df[df['Category']=='ham']
df_spam = df[df['Category']=='spam']

In [7]:
df_ham.shape

(4825, 2)

In [8]:
df_spam.shape

(747, 2)

In [9]:
df_spam_balance = df_spam.sample(df_ham.shape[0],replace=True)
df_spam_balance.shape

(4825, 2)

In [10]:
dataset = pd.concat([df_ham,df_spam_balance])
dataset.shape

(9650, 2)

In [11]:
dataset['Category'].value_counts()

ham     4825
spam    4825
Name: Category, dtype: int64

In [12]:
dataset.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...


In [13]:
dataset['Mail'] = dataset['Category'].apply(lambda x:1 if x=='spam' else 0)

In [14]:
dataset.head()

,Category,Message,Mail
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
6,ham,Even my brother is not like to speak with me. ...,0


In [15]:
dataset['Mail'].value_counts()

0    4825
1    4825
Name: Mail, dtype: int64

In [16]:
# Split the data into train and test

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(dataset['Message'],dataset['Mail'],test_size =0.2,random_state =101, stratify=dataset['Mail'])

In [17]:
!pip install tensorflow_text
#targeting the text data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.4 MB/s eta 0:00:00


In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
print("Number of GPU's available:",len(tf.config.list_physical_devices('GPU')))

Number of GPU's available: 1


In [20]:
import tensorflow_hub as hub
import tensorflow_text as text

# BERT Transformer Model

In [21]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# just for reference ***************************

In [22]:
def get_sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']


In [23]:
get_sentence_embedding([
    "Hello Everyone, Hope you are enjoying Transformer Model",
    "Please practice a lot to acquire Advanced knowledge"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8847765 , -0.43212557, -0.93838775, ..., -0.8243895 ,
        -0.6966927 ,  0.87514377],
       [-0.8971292 , -0.3963175 , -0.605814  , ..., -0.53190994,
        -0.73792297,  0.90347326]], dtype=float32)>

In [24]:
# get embedding vectors for few sample examples

sample = get_sentence_embedding(['mango','banana','Jagan','pawan','flight'])

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
cosine_similarity([sample[0]],[sample[1]])

array([[0.99540985]], dtype=float32)

In [27]:
cosine_similarity([sample[0]],[sample[4]])

array([[0.97404057]], dtype=float32)

In [28]:
cosine_similarity([sample[0]],[sample[3]])

array([[0.9703381]], dtype=float32)

In [29]:
cosine_similarity([sample[2]],[sample[3]])

array([[0.9080776]], dtype=float32)

In [30]:
#********************************************

In [31]:
 from keras.layers import *
 from keras.models import Model

In [34]:
# Building Model
text_input = keras.layers.Input(shape=(),dtype =tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural Network Layers

x1 = keras.layers.Dropout(0.1, name ='dropout')(outputs ['pooled_output'])
x2 = keras.layers.Dense(1,activation = 'sigmoid',name ='output')(x1)

model = keras.Model(inputs=[text_input],outputs =[x2])

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [36]:
len(x_train)

7720

In [37]:
len(x_test)

1930

In [38]:
metrics = [keras.metrics.BinaryAccuracy(name = 'accuracy'),keras.metrics.Precision(name = 'Precision'),keras.metrics.Recall(name ='recall')]
model.compile(optimizer = 'adam',loss ='binary_crossentropy',metrics = metrics)

In [40]:
model.fit(x_train,y_train,batch_size =32,epochs = 10,validation_data =(x_test,y_test))

Epoch 1/10
242/242 [==============================] - 115s 435ms/step - loss: 0.4463 - accuracy: 0.8250 - Precision: 0.8256 - recall: 0.8241 - val_loss: 0.3000 - val_accuracy: 0.9228 - val_Precision: 0.8992 - val_recall: 0.9523
Epoch 2/10
242/242 [==============================] - 104s 431ms/step - loss: 0.2843 - accuracy: 0.9023 - Precision: 0.8924 - recall: 0.9150 - val_loss: 0.2366 - val_accuracy: 0.9301 - val_Precision: 0.9521 - val_recall: 0.9057
Epoch 3/10
242/242 [==============================] - 104s 430ms/step - loss: 0.2338 - accuracy: 0.9242 - Precision: 0.9187 - recall: 0.9308 - val_loss: 0.1920 - val_accuracy: 0.9508 - val_Precision: 0.9448 - val_recall: 0.9575
Epoch 4/10
242/242 [==============================] - 104s 430ms/step - loss: 0.2068 - accuracy: 0.9370 - Precision: 0.9319 - recall: 0.9430 - val_loss: 0.1708 - val_accuracy: 0.9565 - val_Precision: 0.9490 - val_recall: 0.9648
Epoch 5/10
242/242 [==============================] - 104s 430ms/step - loss: 0.1842 - a